In [16]:
import os
import xmltodict
import csv

# CSVファイルの出力先
output_file = "output_data.csv"

# 抽出したい文字列
user_name_to_extract = "HANBAI"
timestamp_to_extract_s = "2023-12-13T01:00:25.744452Z"
timestamp_to_extract_e = "2023-12-13T01:09:50.744452Z"

# 抽出する要素の指定
#elements_to_extract = ['Session_Id','EntryId','DB_User', 'Userhost','Extended_Timestamp','Comment_Text','DBID']
elements_to_extract = ['Session_Id','EntryId','DB_User', 'Userhost','Extended_Timestamp','Comment_Text','DBID','Sql_Text']

# CSVファイルへの追記関数
def append_to_csv(extracted_data, processed_rows):
    with open(output_file, 'a', newline='') as csvfile:
        fieldnames = elements_to_extract
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # ヘッダーの追記
        if csvfile.tell() == 0:  
            writer.writeheader()

        # CSVファイルへの書き込み
        row = {}
        for field in fieldnames:
            row[field] = extracted_data.get(field, '')  
        writer.writerow(row)

# 処理した行数をカウントする変数
processed_rows = 0

# 指定されたフォルダ内のすべてのXMLファイルを処理
folder_path = "./data"
for filename in os.listdir(folder_path):
    if filename.endswith(".xml"):
        # XMLファイルを読み込み、XML_DATAに追加
        with open(os.path.join(folder_path, filename), "r") as xml_file:
            XML_DATA = xml_file.read()

        parsed = xmltodict.parse(XML_DATA)

        print('#############',filename)
        # 指定された文字列の場合にデータを抽出する
        for rdata in parsed['Audit']['AuditRecord']:
            extracted_data = {}
            for element in elements_to_extract:
                extracted_data[element] = rdata.get(element, '')  

            if (rdata['DB_User'] == user_name_to_extract) and \
                rdata['Extended_Timestamp'] >= timestamp_to_extract_s and \
                rdata['Extended_Timestamp'] <= timestamp_to_extract_e:
                # CSVファイルへの追記
                append_to_csv(extracted_data, processed_rows)
                processed_rows += 1  

                #print(f"{', '.join(elements_to_extract)} data for {user_name_to_extract} from {filename} has been appended to {output_file}")

        # 処理した行数を出力
        print(f"file {processed_rows} rows have been appended to {output_file}.")

print(f"Total {processed_rows} rows have been appended to {output_file}.")
# 処理した行数もCSVファイルに追記
with open(output_file, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([f"Processed Rows:{processed_rows} finished!"])


############# DWH_ora_23288_20231213100338343742161532.xml
file 8 rows have been appended to output_data.csv.
############# DWH_ora_24378_20231213100830368940366409.xml
file 315 rows have been appended to output_data.csv.
############# DWH_ora_24203_20231213100736371452344179.xml
file 330 rows have been appended to output_data.csv.
Total 330 rows have been appended to output_data.csv.


In [17]:
import pandas as pd
# CSVファイルの読み込み
df = pd.read_csv("output_data.csv")
df = df.reset_index()
df = df.rename(columns = {"index":"parsedlog_num"})
#df.info(),df.head(10)

## date cleazing

In [18]:
#結合キーparsedlog_numとSQLのみ抽出したdf2を生成
#データクレンジングしたdf3を生成
df2 = df.loc[:,["parsedlog_num","Sql_Text"]]
#print(df2.head())
#print(df2['Sql_Text'].isnull().any())
#print(df2['Sql_Text'].isnull())
#print(df2['Sql_Text'].dropna(how='any'))
df3 = df2.dropna(how='any')
#df3.head(10)
# dfs = df3['Sql_Text']
# print(dfs.isnull().any())
# dfs.info(),dfs

# for i,v in dfs.items():
#    print(i,v)

## 正規表現で取得
- スキーマ名とエイリアス無しのスキーマ名.テーブル名も取得可能
- where区なくても取得可能
- 最後に"A1"'の場合は、エイリアス取得できない。"A1" 'なら取得可能。
    sql99 = 'SELECT COUNT(*) FROM "全社大_自社需要_ビＳ"."料金内訳履歴" "A1"'
- 解釈
table_names_with_aliases = re.findall(r'(?:\s*"([^"]+)"\s*\.)?\s*"([^"]+)"(?:\s+"([^"]+)")?', from_section)
(?: ... )? は、グループ全体がオプションであることを示します。すなわち、グループ内のパターンが0回または1回の繰り返しとなります。
": ダブルクォート（"）をマッチします。([^"]+): ダブルクォート以外の文字列（"を含まない）をキャプチャします。[^"]はダブルクォート以外の文字を表し、+は1回以上の繰り返しを表します。
\s*: 0個以上の空白文字（空白、タブ、改行）をマッチします。
\.: ピリオド（.）をマッチします。
\s*: 0個以上の空白文字（空白、タブ、改行）をマッチします。
"([^"]+)": 同様にダブルクォートで囲まれた文字列をキャプチャします。
(?:\s+"([^"]+)")?: オプションの部分グループです。0回または1回の繰り返しを表します。これは、スペースに続いてダブルクォートで囲まれた文字列をキャプチャします。



In [19]:
#SQL解析　閲覧テーブル抽出
import re

c = 1
hist_tables = []
#print(df3.info(),df3.head())
for row in df3.itertuples(): 
    #print(row[0],row[1],row[2])
    #print("i====",i,"v=====",type(v),v.info())
    sql_text = row[2]
    #sql_text = df['Sql_Text'][1]
    # sql_text = 'SELECT * FROM \
    #  "全社大_自社需要_ビＳ"."需給単位" "A3"\
    #  , "全社大_自社需要_ビＳ"."需給契約" "A2"\
    #  ,"全社大_自社需要_ビＳ"."料金内訳履歴" "A1" \
    #  WHERE = 1;'
    #sql_text = 'SELECT DISTINCT "A1"."営業所コード","A1"."需給単位一連番号","A1"."需給単位枝番号","A1"."需給契約一連番号","A3"."契約種別コード","A2"."業種コード","A2"."使用者_お客さま名漢字","A2"."使用者_お客さま名カナ","A3"."使用場所_住所","A3"."使用場所_住所コード","A3"."標準住所補足_街区_番地","A3"."標準住所補足_番号_枝","A3"."標準住所補足_アパートビル名","A3"."標準住所補足_部屋番号","A3"."非標準住所補足","A1"."支払方法コード","A1"."支払者番号","A2"."支払お客さま番号","A1"."支払者肩書漢字名","A1"."支払者肩書カナ名","A1"."支払者漢字名","A1"."支払者カナ名","A1"."郵便番号","A1"."住所コード","A1"."支払場所漢字","A1"."住所補足街区漢字","A1"."住所補足枝番漢字","A1"."アパートビル名","A1"."部屋番号","A1"."非標準住所補足漢字","A1"."金融機関コード","A1"."店舗コード","A1"."預金種別コード","A1"."口座番号","A1"."単価パターンコード" FROM "sasa2全社大_自社需要_ビＳ"."sasa2需給単位","sasa全社大_自社需要_ビＳ"."sasa需給単位" "sasaA3","全社大_自社需要_ビＳ"."需給単位" "A3","全社大_自社需要_ビＳ"."需給契約" "A2","全社大_自社需要_ビＳ"."料金内訳履歴" "A1" WHERE "A3"."営業所コード"="A2"."営業所コード" AND "A3"."需給単位一連番号"="A2"."需給単位一連番号" AND "A3"."需給単位枝番号"="A2"."需給単位枝番号" AND "A2"."営業所コード"="A1"."営業所コード" AND "A2"."需給単位一連番号"="A1"."需給単位一連番号" AND "A2"."需給単位枝番号"="A1"."需給単位枝番号" AND "A2"."需給契約一連番号"="A1"."需給契約一連番号" AND ("A3"."需給単位一連番号"=\'61975180\' AND "A3"."需給単位枝番号"=\'1\' OR "A3"."需給単位一連番号"=\'61975201\' AND "A3"."需給単位枝番号"=\'2\' OR "A3"."需給単位一連番号"=\'61975201\' AND "A3"."需給単位枝番号"=\'3\' OR "A3"."需給単位一連番号"=\'61975203\' AND "A3"."需給単位枝番号"=\'1\' OR "A3"."需給単位一連番号"=\'61975203\' AND "A3"."需給単位枝番号"=\'2\' OR "A3"."需給単位一連番号"=\'62300361\' AND "A3"."需給単位枝番号"=\'1\' OR "A3"."需給単位一連番号"=\'20502220\' AND "A3"."需給単位枝番号"=\'1\' OR "A3"."需給単位一連番号"=\'20502220\' AND "A3"."需給単位枝番号"=\'2\' OR "A3"."需給単位一連番号"=\'67853548\' AND "A3"."需給単位枝番号"=\'1\' OR "A3"."需給単位一連番号"=\'67853548\' AND "A3"."需給単位枝番号"=\'2\' OR "A3"."需給単位一連番号"=\'66861640\' AND "A3"."需給単位枝番号"=\'1\' OR "A3"."需給単位一連番号"=\'66861640\' AND "A3"."需給単位枝番号"=\'2\' OR "A3"."需給単位一連番号"=\'67873728\' AND "A3"."需給単位枝番号"=\'2\' OR "A3"."需給単位一連番号"=\'64691547\' AND "A3"."需給単位枝番号"=\'1\' OR "A3"."需給単位一連番号"=\'52972553\' AND "A3"."需給単位枝番号"=\'1\' OR "A3"."需給単位一連番号"=\'64729950\' AND "A3"."需給単位枝番号"=\'1\' OR "A3"."需給単位一連番号"=\'64729951\' AND "A3"."需給単位枝番号"=\'1\' OR "A3"."需給単位一連番号"=\'64729952\' AND "A3"."需給単位枝番号"=\'1\' OR "A3"."需給単位一連番号"=\'10364800\' AND "A3"."需給単位枝番号"=\'1\' OR "A3"."需給単位一連番号"=\'10364809\' AND "A3"."需給単位枝番号"=\'2\' OR "A3"."需給単位一連番号"=\'10817789\' AND "A3"."需給単位枝番号"=\'1\' OR "A3"."需給単位一連番号"=\'10817790\' AND "A3"."需給単位枝番号"=\'1\') AND "A1"."料金計算年月"=TO_CHAR(ADD_MONTHS(SYSDATE@!,(-1)),\'yyyymm\') AND "A3"."契約廃止コード"=\'0\' ORDER BY CASE  WHEN ("A3"."契約種別コード"=05 OR "A3"."契約種別コード"=07 OR "A3"."契約種別コード"=09 OR "A3"."契約種別コード"=10 OR "A3"."契約種別コード"=11 OR "A3"."契約種別コード"=19 OR "A3"."契約種別コード"=31 OR "A3"."契約種別コード"=83 OR "A3"."契約種別コード"=85) THEN 1 ELSE 2 END ,"A3"."使用場所_住所","A3"."使用場所_住所コード","A3"."標準住所補足_街区_番地","A3"."標準住所補足_番号_枝","A3"."標準住所補足_アパートビル名","A3"."標準住所補足_部屋番号","A3"."非標準住所補足","A1"."営業所コード","A1"."需給単位一連番号","A1"."需給単位枝番号"'
    #sql_text = 'select * FROM "全社大"."sasa" '
    #sql_text = 'select * FROM "sasa" :'
    # sql_text = 'select * FROM "sasa" "A2" :'

    from_index = sql_text.find('FROM')
    #print('from_index',from_index)
    where_index = sql_text.find('WHERE') if 'WHERE' in sql_text else sql_text.find(';')
    #print('where_index',where_index)
    from_section = sql_text[from_index:where_index]
    #print('from_section',from_section)

    #スキーマ名.テーブル名は取得可能
    table_names_with_aliases = re.findall(r'(?:\s*"([^"]+)"\s*\.)?\s*"([^"]+)"(?:\s+"([^"]+)")?', from_section)

    #print('table_names_with_aliases',table_names_with_aliases)

    temp_tables = []
    for schema, table, alias in table_names_with_aliases:
        temp_tables.append((row[1],schema, table, alias))
        hist_tables.append((row[1],schema, table, alias))

    #print(temp_tables)
    #print("抽出されたテーブル:"+f"ループのカウント{c}")
#     for i, (row1,schema, table, alias) in enumerate(temp_tables, start=1):
#         print(f"parsedlog_num='{row1}' テーブル {i}: スキーマ名='{schema}', テーブル名='{table}', エイリアス名='{alias}'")
    
    c = c+1
        

In [20]:
#抽出した閲覧テーブルを元表へ結合しdf4を生成
df_hist = pd.DataFrame(hist_tables,columns=['parsedlog_num', 'schema', 'table','alias'])
print(df_hist)

#完全外部結合でも差分は出ず、parsedlog_numで全行一致。
df4 = pd.merge(df, df_hist, on='parsedlog_num', how='outer')
#print(df4.info())
#df4.tail(30)
#df4.head(30)


     parsedlog_num       schema   table alias
0                1  全社大_自社需要_ビＳ    需給単位    A3
1                1  全社大_自社需要_ビＳ    需給契約    A2
2                1  全社大_自社需要_ビＳ  料金内訳履歴    A1
3                2  全社大_自社需要_ビＳ    需給単位    A3
4                2  全社大_自社需要_ビＳ    需給契約    A2
..             ...          ...     ...   ...
357            327  全社大_自社需要_ビＳ    需給単位    A3
358            327  全社大_自社需要_ビＳ    需給契約    A2
359            327  全社大_自社需要_ビＳ  料金内訳履歴    A1
360            328  全社大_自社需要_ビＳ  料金内訳履歴      
361            329  全社大_自社需要_ビＳ  料金内訳履歴      

[362 rows x 4 columns]


In [22]:
df5 = df4.dropna(how='any')
#df5.head(10)

In [21]:
import plotly.express as px

# Plotlyを使用してデータをプロット
# fig = px.scatter(df, x='Extended_Timestamp', y='EntryId', title=f"count(EntryId)",color='Session_Id')
fig = px.scatter(df4, x='Extended_Timestamp', y=df4.index, title=f"count(df4.index)",color='Session_Id',hover_name="table", hover_data=["DB_User","Userhost","schema","table"])
fig.show()


In [23]:
import plotly.express as px

# Plotlyを使用してデータをプロット
fig = px.treemap(df5, path=["table","schema","Session_Id"],values= df5.index,hover_name="Userhost", hover_data=["DB_User","Userhost","schema","table"])
#fig = px.treemap(df5, path=["Session_Id","table"],values =df5.index,hover_name="Userhost", hover_data=["DB_User","Userhost","schema","table"])

fig.show()